#### 31/08/2025

In [ ]:
SELECT 
    max(salary)    
FROM employees
WHERE salary <
(
    SELECT 
        max(salary)
    FROM employees
)

In [ ]:
SELECT 
    *    
FROM employees
WHERE salary >
(
    SELECT 
        avg(salary)
    FROM employees
)

In [ ]:
-- Using CTE and window functions
WITH nth_sal 
AS 
(
    select 
        *,
        row_number() over(order by salary desc) as highsal
    from employees
)

select * from nth_sal where highsal = 3;

-- Using subquery and window functions

select 
    *
from 
(
    select 
        *,
        row_number() over(order by salary desc) as highsal
    from employees
) as highsal
where highsal = 2;

-- Without using CTE, subquery or ranking function

select * from employees order by salary desc limit 1 offset 1 ; -- here, we get the 2nd highest and offset value can be altered depend on nth value.

In [ ]:
-- using count

select 
    emp_id, -- column names can be added/altered as per requirement
    count(1) as dup_count
from employees
group by emp_id
having count(1) > 1;

-- using CTE and windowing functions

with dup_records
as
(
    select
        *,
        row_number() over(partition by firstname, lastname order by emp_id) as dup_order
    from employees
    
)

select * from dup_records where dup_order > 1;

In [ ]:
DELETE FROM employees
WHERE emp_id NOT IN 
(
    SELECT MIN(emp_id)
    FROM employees
    GROUP BY firstname, lastname
);


In [ ]:
-- using CTE

with same_sal
as
(
    select salary, count(1) as sal_count from employees
    group by salary 
    having count(1) > 1
)
select * from employees
where salary in (select salary from same_sal);

-- using window functions

SELECT *
FROM (
    SELECT e.*, COUNT(*) OVER(PARTITION BY salary) AS sal_count
    FROM employees e
) t
WHERE sal_count > 1;

-- using self join

SELECT e1.*
FROM employees e1
JOIN employees e2 
  ON e1.salary = e2.salary 
 AND e1.emp_id <> e2.emp_id;

In [ ]:
--straightforward

SELECT *
FROM employees
WHERE DATEDIFF(day, hiredate, CURRENT_DATE) < 30;


--using subquery (optional)

select * from
(
    select *, datediff(day, hiredate, current_date()) as empie from employees
) as datedifftable
where empie < 30
order by empie asc;


--using window functions

with hirediff
as 
(
    select e.*, datediff(day, hiredate, current_date()) as empie from employees e
)

select e.* from hirediff as e where empie < 30;

In [ ]:
-- using window functions
with
sal_ranked
as
(
    select
        *,
        dense_rank() over(order by salary desc) as sal_rank
    from employees
)
select 
    distinct salary 
from 
    sal_ranked 
where 
    sal_rank <=3
order by 
    salary desc

In [ ]:
-- using intersect

select * from employees
intersect
select * from employees;

-- using EXISTS

SELECT *
FROM employees e
WHERE EXISTS (
    SELECT 1
    FROM employees e1
    WHERE e1.emp_id = e.emp_id);

#### 02/09/2025

In [ ]:
-- How would you find all the managers who do not manage any employees?

SELECT m.emp_id, m.firstname AS manager_name
FROM employees m
LEFT JOIN employees e
  ON e.manager_id = m.emp_id
WHERE e.emp_id IS NULL;


In [ ]:
-- Write a SQL query to display the departments with more than 10 employees.

select 
    d.dept_id,
    d.dept_name,
    count(e.emp_id) as dept_emp_count
from 
    dept d
join 
    employees e 
on 
    e.dept_id = d.dept_id 
group by
    d.dept_id, d.dept_name
having count(e.emp_id) > 10

In [ ]:
-- How do you find the names of employees that start with ‘A’?

select
    firstname,
    middlename,
    lastname
from 
    employees
where firstname ILIKE 'a%'

In [ ]:
-- Write a query to retrieve the employee details along with their manager details.

select
    e.emp_id as employee_id,
    m.manager_id as manager_id,
    e.firstname as employeename,
    m.firstname as managername
from
    employees e
left join
    employees m
on e.manager_id = m.emp_id

In [ ]:
-- Write a SQL query to get the employee with the highest salary per department.

select
    firstname,
    lastname,
    salary,
    dept_id
from
(
    select 
        *,
        row_number() over(partition by dept_id order by salary desc) as dept_sal
    from employees  
) as emp_high_sal
where dept_sal = 1
order by dept_id asc

In [ ]:
-- Write a query to find the total salary paid to each department.

select
    dept_id,
    sum(salary) as total_dept_sal
from
    employees
group by dept_id
order by total_dept_sal desc

#### 03/09/2025

In [ ]:
-- How would you find employees who work in more than one department?
select
    emp_id,
    count(distinct dept_id) as emp_work_dept_count
from
    employees
group by
    emp_id
having
    emp_work_dept_count > 1
order by
    emp_work_dept_count desc

In [ ]:
-- Write a query to fetch the employee details along with their salary in ascending order.

select
    emp_id,
    firstname,
    lastname,
    salary
from 
    employees
order by 
    salary asc

In [ ]:
-- Write a SQL query to find all employees whose department is not in the IT department.

select
    e.emp_id,
    e.firstname,
    e.lastname,
    d.dept_name
from    
    employees e
join
    dept d
on e.dept_id = d.dept_id
where d.dept_name NOT ILIKE 'IT%' 

In [ ]:
-- How do you retrieve the last record from a table?

--“In SQL, rows are unordered, so there’s no concept of a ‘last record’ without an ORDER BY. If we have a meaningful column like created_at or an auto-increment id, we can use ORDER BY … DESC LIMIT 1 to fetch the latest.”

select
    *
from employees
order by hiredate desc limit 1

In [ ]:
-- Write a query to count the number of employees in each department.

select
    d.dept_id,
    d.dept_name,
    count(e.emp_id) as num_of_employees
from 
    dept d
join
    employees e
on 
    d.dept_id = e.dept_id
group by
    d.dept_id, d.dept_name
order by 
    num_of_employees desc

### 04/09/2025

In [ ]:
-- Write a query to fetch the current date and time in SQL.

select 
    current_date() as current_sql_date,
    current_time() as current_sql_time

In [ ]:
-- Write a query to find employees whose salary is between 50,000 and 100,000

select
    firstname,
    lastname,
    salary
from 
    employees
where
    salary between 50000 and 100000

In [ ]:
-- How do you retrieve the first three characters of a string in SQL?

select 
    left(firstname, 3) as firstname_first_three,
    left(lastname, 3) as lastname_first_three
from
    employees

In [ ]:
-- How do you retrieve the third highest salary without using TOP, LIMIT, or OFFSET?

WITH
third_high_sal
AS
(
    select
        *,
        dense_rank() over(order by salary desc) as sal_rank
    from
        employees
)
select 
    distinct salary
from
    third_high_sal
where
    sal_rank = 3

In [ ]:
-- Write a query to calculate the cumulative sum of salaries ordered by hire date.

SELECT
    emp_id,
    hiredate,
    salary,
    SUM(salary) OVER(ORDER BY hiredate) AS cum_sal
FROM employees
order by hiredate;

### 04/09/2025

In [ ]:
-- How do you write a query to generate a running row number for each employee?

select
    *,
    row_number() over(order by emp_id) as running_row_num
from employees

In [ ]:
-- Write a query to fetch the top 2 salaries in each department.

WITH
toptwosal
as
(
    select
        *,
        dense_rank() over(partition by dept_id order by salary desc) as dept_toptwo_sal
    from
        employees
)
select 
    firstname,
    lastname,
    dept_id,
    salary
from
    toptwosal where dept_toptwo_sal <= 2
order by dept_id, salary desc

In [ ]:
-- Write a query to return departments that don’t have any employees.

select
    d.dept_id,
    d.dept_name,
    count(e.emp_id) as emp_count
from
    dept d
left join employees e
on d.dept_id = e.dept_id
group by d.dept_id, d.dept_name
having count(e.emp_id) = 0

In [ ]:
-- Write a query to list employees who were hired in the same year.
WITH 
hireyear
as
(
select
    firstname,
    lastname,
    year(hiredate) as hired_year,
    count(emp_id) over(partition by year(hiredate)) as part_hiredate_year
from   
    employees
)
select * from hireyear
where part_hiredate_year > 1
order by hired_year 

In [ ]:
-- Write a query to find the maximum salary in each department without using GROUP BY.

With maxsal
as 
(
    select
        *,
        row_number() over(partition by dept_id order by salary desc) as dept_sal
    from employees
)
select
    firstname,
    lastname,
    dept_id,
    salary
from maxsal
where dept_sal = 1
order by dept_id

In [ ]:
-- How do you find employees whose hire date is the same as their manager’s hire date?

select
    e.firstname as empname,
    m.firstname as managername,
    e.hiredate as emp_hiredate,
    m.hiredate as man_hiredate
from
    employees e
left join 
    employees m
on 
    e.manager_id = m.emp_id
where e.hiredate = m.hiredate

In [ ]:
-- Write a query to fetch employees with salaries greater than the salary of their manager.

select
    e.firstname as employee_name,
    m.firstname as manager_name,
    e.salary as employee_salary,
    m.salary as manager_salary
from
    employees e
join
    employees m
on 
    e.manager_id = m.emp_id
where e.salary > m.salary

In [ ]:
-- Write a query to fetch employees whose salary equals the average salary in their department.

WITH
avg_sal
as
(
    select
        *,
        round(avg(salary) over(partition by dept_id),0) as dept_avg_sal
    from employees
)
select 
    firstname,
    lastname,
    dept_id,
    salary
from 
    avg_sal
where 
    salary = dept_avg_sal
order by 
    dept_id

In [ ]:
-- Write a query to calculate year-over-year growth in sales.

select
    *,
    year(hiredate) as year_hiredate,
    sum(salary) over(partition by year(hiredate)) as ranke
    from employees
order by year_hiredate asc

In [ ]:
-- Find the 5 most recently hired employees in each department.
WITH
recenthire
AS
(
    select 
        *,
        dense_rank() over(partition by dept_id order by hiredate desc) as dept_hire
    from 
        employees
)
select 
    emp_id,
    firstname,
    lastname,
    dept_id,
    hiredate
from recenthire
where dept_hire <=5
order by dept_id;

In [ ]:
-- List departments where the average salary exceeds the overall company average.

WITH
dept_avg_sal
as
(
    select 
        dept_id,
        avg(salary) as dept_avg
    from employees
    group by dept_id
)
select 
    distinct dept_id 
from 
    dept_avg_sal
where dept_avg > (
            select
                round(avg(salary),0) as company_avg
            from
                employees
            )

In [ ]:
-- Retrieve employees who earn more than their manager.

select
    e.firstname as employee_name,
    m.firstname as manager_name,
    e.salary as employee_salary,
    m.salary as manager_salary
from
    employees e
join   
    employees m
on 
    e.manager_id = m.emp_id
where
    e.salary > m.salary

In [ ]:
-- Find employees who were hired before their manager.
select
    e.emp_id as employee_id,
    e.firstname as employee_name,
    m.firstname as manager_name,
    e.hiredate as employee_hire_date,
    m.hiredate as manager_hire_date
from
    employees e
    join employees m on e.manager_id = m.emp_id
where
    manager_hire_date > employee_hire_date
order by
    employee_id asc

In [ ]:
-- Identify departments where no employee has a salary above 70,000.
with dept_wise_sal as (
    select
        dept_id,
        max(salary) as dept_max_sal
    from
        employees
    group by
        dept_id
)
select
    *
from
    dept_wise_sal
where
    dept_max_sal <= 70000;

In [ ]:
-- Find employees whose birthday falls in the current month.
select
    emp_id,
    firstname as employeename,
    month(hiredate) as birth_month
from
    employees
where
    month(hiredate) = month(current_date())
order by
    emp_id

In [ ]:
-- Retrieve the youngest employee in each department.
with dept_wise_youngest as (
    select
        emp_id,
        firstname,
        lastname,
        dept_id,
        DOB,
        dense_rank() over(partition by dept_id order by dob desc) as dob_dept_order
    from
        employees
)
select 
    * 
from 
    dept_wise_youngest
where
    dob_dept_order = 1
order by
    dept_id

In [ ]:
-- Find the oldest 3 employees in the company.
select 
    emp_id,
    firstname,
    lastname,
    DOB
from
    employees
order by
    DOB
limit 3

In [ ]:
-- List employees who share both the same manager and department.
select
    e.emp_id as employee_id,
    e.firstname as employee_name,
    e.manager_id manager_id,
    m.firstname as manager_name,
    e.dept_id as employee_department
from
    employees e join employees m
on
    e.manager_id = m.emp_id
where
    (e.manager_id, e.dept_id) IN
(
    select manager_id, dept_id
    from employees
    group by manager_id, dept_id
    having count(emp_id) >1
)

In [ ]:
-- Find departments where the manager of at least one employee is not the highest-paid person in that department.
WITH dept_max AS (
    SELECT 
        dept_id,
        MAX(salary) AS dept_max_sal
    FROM employees
    GROUP BY dept_id
)
SELECT 
    DISTINCT e.dept_id
FROM employees e
JOIN employees m ON e.manager_id = m.emp_id
JOIN dept_max d ON e.dept_id = d.dept_id
WHERE m.salary < d.dept_max_sal;

In [ ]:
-- Get employees who joined before the department was created.

select 
    e.firstname,
    e.hiredate,
    e.dept_id,
    d.created_date
from employees e
join dept d
on e.dept_id = d.dept_id
where e.hiredate < d.created_date;

In [ ]:
-- Identify employees who work in departments located on floor number 2.

select 
    e.emp_id,
    e.firstname,
    e.lastname,
    d.floor_number
from 
    employees e 
join
    dept d
on 
    e.dept_id = d.dept_id
where 
    d.floor_number = 2

In [ ]:
-- Find employees who have no phone number listed.

select 
    emp_id,
    firstname,
    lastname
from 
    employees
where phone is null

In [ ]:
-- Retrieve employees who have both first and last names starting with the same letter.

select
    emp_id,
    firstname,
    lastname
from
    employees
where
   lower(left(firstname,1)) = lower(left(lastname,1))

In [ ]:
-- List employees who have a middle name but no manager assigned.

select
    emp_id,
    firstname,
    middlename,
    lastname,
    manager_id
from
    employees
where
    middlename is not null and manager_id is null

In [ ]:
-- Find departments where the total salary spend is more than the budget.

select
    e.dept_id,
    d.budget,
    sum(e.salary) as dept_total_sum
from
    employees e
join
    dept d
on 
    e.dept_id = d.dept_id
group by 
    e.dept_id, d.budget
having 
    dept_total_sum > d.budget

In [ ]:
-- List employees who do not belong to any department.

select
    emp_id,
    firstname,
    lastname,
    dept_id
from
    employees
where
    dept_id is null

In [ ]:
-- Identify managers who manage more than 5 employees.

select
    m.manager_id,
    m.firstname as manager_name,
    count(e.emp_id) as emp_count
from
    employees m
join 
    employees e
on
    m.emp_id = e.manager_id
group by
    m.manager_id, m.firstname
having
    count(e.emp_id) > 5
order by
    emp_count desc

In [ ]:
-- Find employees who report directly or indirectly to a specific manager.

with recursive reports(emp_id, firstname, lastname, manager_id) as (
    -- Direct reports
    select emp_id, firstname, lastname, manager_id
    from employees
    where manager_id = 177368

    union all

    -- Indirect reports
    select e.emp_id, e.firstname, e.lastname, e.manager_id
    from employees e
    join reports r on e.manager_id = r.emp_id
)
select *
from reports
order by manager_id, emp_id;

In [ ]:
-- Get the average age of employees by department.
select
    dept_id,
    round(avg(datediff(year,dob, current_date())),1) as dept_avg_age
from
    employees
group by
    dept_id
order by
    dept_id

In [ ]:
-- Find employees whose salary is in the top 10% of their department.

select 
    *
from
(
    select firstname, lastname, dept_id,
    percent_rank() over(partition by dept_id order by salary desc) as pr 
    from employees
) t
where pr <= 0.1
order by dept_id

/*
select *
from (
    select *,
           ntile(10) over (partition by dept_id order by salary desc) as nt
    from employees
) t
where nt = 1
order by dept_id;
*/

In [ ]:
-- Retrieve employees who have the same last name but work in different departments.

select distinct
    e1.lastname,
    e2.lastname,
    e1.dept_id,
    e2.dept_id
from employees e1
join employees e2 
  on e1.lastname = e2.lastname
 and e1.dept_id != e2.dept_id;

/*select lastname
from employees
group by lastname
having count(distinct dept_id) > 1;*/

In [ ]:
-- Find employees whose hire date and date of birth fall on the same day of the month.

select 
    firstname,
    lastname,
    hiredate,
    dob
from
    employees
where
    day(hiredate) = day(dob)

In [ ]:
-- Identify departments without a head assigned.

select
    dept_id,
    dept_name
from
    dept
where
    head_id is null

In [ ]:
-- List departments where the head’s hire date is after the department’s created date.

-- If there’s no `head_id` in `dept`, the question can’t be answered as stated. Ignoring it is the right move unless schema changes.


In [ ]:
-- Retrieve the difference between the highest and lowest salaries in each department.

select
    dept_id,
    max(salary) - min(salary) as dept_sal_difference  
from
    employees
group by
    dept_id
order by
    dept_id

In [ ]:
-- Find employees whose email domain matches their department location name.

select
    e.dept_id,
    substr e.email,
    d.dept_name
from
    employees e join dept d
on e.dept_id = d.dept_id

In [ ]:
-- List employees who were hired in a leap year.

select *
from employees
where 
    (year(hiredate) % 400 = 0)
    or (year(hiredate) % 4 = 0 and year(hiredate) % 100 != 0);

In [ ]:
-- Retrieve employees with palindromic first names.

select 
    firstname
from
    employees
where firstname = reverse(firstname)

In [ ]:
-- Identify the top 3 highest-paying departments by average salary.

select
    dept_id,
    round(avg(salary),0) as dept_avg_sal
from
    employees
group by 
    dept_id
order by
    dept_avg_sal desc
limit 3

In [ ]:
-- Find employees whose salary ranks 3rd within their department.

select
    *
from
(
    select emp_id, firstname,lastname, salary, dept_id,
    dense_rank() over(partition by dept_id order by salary desc) as dept_sal
    from employees
) t
where dept_sal = 3

In [ ]:
-- List departments where no employees were hired in the last 2 years.

select
    distinct edept_id
from
    employees
where
    hiredate in (select hiredate from employees where datediff(year,hiredate, current_date()) = 2)
and
    e1.dept_id 
    

In [ ]:
-- Retrieve employees with duplicate phone numbers.
-


In [ ]:
-- Find the employee(s) with the longest full name.



In [ ]:
-- Get departments where the employee count is less than 3.

